#LangChain을 이용한 댓글 분류 자동화 시스템 구현
이 코드에서는 댓글 데이터를 긍정적 또는 부정적으로 분류하는 간단한 시스템을 구현합니다. 이 시스템은 AI 모델을 사용하여 각 댓글을 분석하고, 긍정적이면 `1`, 부정적이면 `-1`을 출력합니다. 이 코드는 OpenAI API를 사용하며, 주어진 데이터를 기반으로 정확도를 계산합니다.


LangChain 기반 댓글 감성 분석 자동화 시스템 - 큰 그림부터 이해하기


✅ 1. 이 코드가 무엇을 하는지부터 간단히 설명합니다
이 코드는 우리가 평소에 남기는 댓글(예: 리뷰나 피드백)이 긍정적인지, 부정적인지를 AI가 자동으로 분류해주는 시스템입니다.

예: "이 제품 정말 좋아요!" → 긍정(1)

예: "진짜 별로였어요." → 부정(-1)

우리는 AI에게 리뷰 하나하나를 보여주고, AI가 그것이 긍정인지 부정인지 판단하게 시키는 것입니다.



✅ 2. 이 시스템은 어떻게 동작하나요? (큰 흐름)
이 시스템은 크게 4단계로 나눠집니다:

- 리뷰 데이터를 준비합니다

  엑셀 파일에 있는 실제 리뷰(문장)들과 정답(긍정:1, 부정:-1)을 읽어옵니다.

-- AI에게 역할을 알려줍니다 (시스템 프롬프트)

  "너는 리뷰 분석 전문가야! 리뷰가 긍정이면 1, 부정이면 -1을 답해줘." 라고 AI에게 미리 알려줍니다.

- 리뷰 하나씩 AI에게 분석하도록 넘깁니다

  각 리뷰를 AI에게 주고, "이건 어때?" 하고 물어봅니다.

- AI의 판단과 실제 정답을 비교해서 정확도를 계산합니다

  AI의 답이 정답과 일치하는지 확인합니다.

  얼마나 맞췄는지 비율을 계산해서 정확도를 구합니다.



✅ 3. 어떤 도구들이 사용되었나요?

- LangChain: AI 모델과 대화하듯 데이터를 처리할 수 있게 도와주는 프레임워크입니다.

- OpenAI 모델 (GPT-3.5, GPT-4 등): AI가 실제로 판단을 내리는 두뇌 역할입니다.

- Excel 파일: 리뷰와 정답이 담긴 데이터 저장소입니다.



✅ 4. 핵심 아이디어 요약
- AI에게 먼저 역할을 부여합니다. (시스템 프롬프트)

- 데이터를 AI에게 하나씩 보여줍니다.

- AI의 판단 결과를 수집합니다.

- 실제 정답과 비교해서 정확도를 계산합니다.



✅ 5. 직관적인 예시
리뷰 내용	AI 출력	실제 정답	맞췄나요?
"완전 만족이에요!"	1	1	✅
"배송이 너무 늦었어요."	-1	-1	✅
"좋긴 한데 아쉬워요."	1	-1	❌

이런 식으로 AI가 얼마나 잘 맞추는지 평가하는 시스템입니다.



✅ 6. 전체 로직 한 문장으로 요약하자면?
"AI에게 리뷰를 읽게 해서 긍정/부정 판단을 시키고, 얼마나 잘 맞추는지 측정하는 자동화된 감성 분석 시스템입니다."

In [2]:
# 필요한 외부 라이브러리들을 설치합니다. LangChain, OpenAI 관련 모듈 포함
!pip install langchain_community cohere openai tiktoken langchain numexpr tabulate llmdantic langchain_openai -q


In [ ]:
# OpenAI 라이브러리 불러오기
import openai
import os

# [포인트] API 키를 환경변수로 등록합니다. (보안 목적)
# API 키는 OpenAI 서비스를 사용할 때, 내 계정을 인증하는 데 사용됩니다.
# os.environ['OPENAI_API_KEY']=
# openai 클라이언트 생성 (현재 버전에서 필수는 아니지만 일부 기능에서 사용 가능)
client = openai.OpenAI()

# 키 등록 확인용 메시지
print("API 키 설정이 완료되었습니다.")

API 키 설정이 완료되었습니다.


## Langchain기반 댓글 감성분석 자동화(Langchain으로 모델만 불러온 경우)

In [4]:
# ======================= 1차 실행: GPT-4o 모델을 이용한 분류 =======================
import pandas as pd
from langchain.chat_models import ChatOpenAI

# [포인트] 엑셀 파일에서 리뷰 데이터를 불러옵니다.
# 'Review' 열에는 댓글 내용, 'Label' 열에는 실제 정답값 (1 또는 -1)이 있습니다.
data = pd.read_excel('./sample_data/reviews.xlsx')
print(data.head())  # 데이터 미리보기 출력

   Num                                             Review  Label
0    1                                   탄소와 예술의 결합 정말 별로     -1
1    2                                       크게 볼 것은 없습니다     -1
2    3  접근성 별로임. 차 있는 사람들에게 추천.\n아이들과 같이 오는 공간.\n이팝나무가...     -1
3    4  이팝나무도서관 예약하고 갔음.\n추운데 관계자 없어서 밖에서 떨면서 어찌저찌 연락되...     -1
4    5                                             왜 만든건지     -1


In [5]:
# 시스템 프롬프트 정의
# [포인트] AI에게 어떤 역할을 수행해야 하는지 명확하게 알려줍니다.
system_prompt = '''
당신은 리뷰 분석 전문가입니다.
댓글을 읽고, 긍정적이면 1, 부정적이면 -1로 분류하세요.
반드시 정수로만 답하세요

---
'''

# 정확도 계산을 위한 변수 초기화
correct_count = 0  # 맞춘 리뷰 개수
total_count = 0    # 전체 리뷰 개수

# [포인트] 사용할 언어모델을 gpt-4o로 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0.1)

# [포인트] 데이터셋을 한 줄씩 반복하면서 AI에게 분류 요청
for i, row in data.iterrows():
    print(f'# {i+1}')
    print(f'리뷰: {row["Review"]}')  # 현재 리뷰 출력

    # 프롬프트 구성: 시스템 메시지와 유저 메시지를 함께 전달
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": row["Review"]}
    ]

    # AI 호출 → 응답받기
    response = llm.invoke(prompt)
    result = response.content
    print(f'모델 출력: {result} / 실제 라벨: {row["Label"]}')

    # [포인트] AI 응답에 '-1'이 포함되어 있으면 부정으로 판단
    if '-1' in result:
        prediction = -1
    else:
        prediction = 1

    # 예측값이 실제 라벨과 같으면 정확도 카운트 증가
    if prediction == row["Label"]:
        correct_count += 1

    total_count += 1  # 전체 리뷰 수 증가

    print()

# 최종 정확도 출력
accuracy = correct_count / total_count
print(f'정확도: {accuracy:.2%}')



# 1
리뷰: 탄소와 예술의 결합 정말 별로
모델 출력: -1 / 실제 라벨: -1

# 2
리뷰: 크게 볼 것은 없습니다
모델 출력: -1 / 실제 라벨: -1

# 3
리뷰: 접근성 별로임. 차 있는 사람들에게 추천.
아이들과 같이 오는 공간.
이팝나무가 진 타이밍에 가서 그런지 볼게 없었음.
작품 전시회가 있어서 본 거 말고는 별로
모델 출력: -1 / 실제 라벨: -1

# 4
리뷰: 이팝나무도서관 예약하고 갔음.
추운데 관계자 없어서 밖에서 떨면서 어찌저찌 연락되어 문 열어주면서 미안하단 말도 없었음
5분쯤 관람했을까? 홍보동영상 찍는다고 찍혀도 괜찮냐 물어서 싫다니 뒷쪽 방 열어줌
그곳에서 50분쯤 책 읽는데 안끝나겠다 싶어서 나옴
전주에 볼거 먹을거 많은데 일부러 시간내서 간게 너무 화가났음
비추
모델 출력: -1 / 실제 라벨: -1

# 5
리뷰: 왜 만든건지
모델 출력: -1 / 실제 라벨: -1

# 6
리뷰: 예술적 감각이 없는건지 모르겠다만 정말 이해도 전혀 안가고 그냥 폐공장일 뿐임
모델 출력: -1 / 실제 라벨: -1

# 7
리뷰: 사진찍기 안좋고 전시물도 없음
모델 출력: -1 / 실제 라벨: -1

# 8
리뷰: 볼게 하나도 없음
모델 출력: -1 / 실제 라벨: -1

# 9
리뷰: 볼게 하나도 없어서 넘 실망스러움
모델 출력: -1 / 실제 라벨: -1

# 10
리뷰: 별다른 경험이 없슴 기대 이하였네요
모델 출력: -1 / 실제 라벨: -1

# 11
리뷰: 너무불친절하세요
모델 출력: -1 / 실제 라벨: -1

# 12
리뷰: 휑한 팔복예술공장
모델 출력: -1 / 실제 라벨: -1

# 13
리뷰: 현대전시는 언젠나 재미없고 이해하기 어려움
모델 출력: -1 / 실제 라벨: -1

# 14
리뷰: 행사 등이 있을때 가는 곳이지 평일 주말에 관광 가는 곳은 아님
모델 출력: -1 / 실제 라벨: -1

# 15
리뷰: 포스터 전시가 그나마 나왔으나 전시 방법이 너무 아쉬움
모델 출력: -1 / 실제 라벨: -1

#

In [6]:
import pandas as pd
from langchain.chat_models import ChatOpenAI

# 동일하게 데이터 불러오기
# data = pd.read_excel('/content/reviews.xlsx')
# print(data.head())

# 동일한 시스템 프롬프트
system_prompt = '''
당신은 리뷰 분석 전문가입니다.
댓글을 읽고, 긍정적이면 1, 부정적이면 -1로 분류하세요.
반드시 정수로만 답하세요

---
'''

# 정확도 초기화
correct_count = 0
total_count = 0

# [포인트] 이번에는 GPT-3.5 모델 사용
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

# 동일한 방식으로 리뷰 하나씩 처리
for i, row in data.iterrows():
    print(f'# {i+1}')
    print(f'리뷰: {row["Review"]}')

    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": row["Review"]}
    ]

    response = llm.invoke(prompt)
    result = response.content
    print(f'모델 출력: {result} / 실제 라벨: {row["Label"]}')

    if '-1' in result:
        prediction = -1
    else:
        prediction = 1

    if prediction == row["Label"]:
        correct_count += 1

    total_count += 1

    print()

# 정확도 출력
accuracy = correct_count / total_count
print(f'정확도: {accuracy:.2%}')

# 1
리뷰: 탄소와 예술의 결합 정말 별로
모델 출력: -1 / 실제 라벨: -1

# 2
리뷰: 크게 볼 것은 없습니다
모델 출력: -1 / 실제 라벨: -1

# 3
리뷰: 접근성 별로임. 차 있는 사람들에게 추천.
아이들과 같이 오는 공간.
이팝나무가 진 타이밍에 가서 그런지 볼게 없었음.
작품 전시회가 있어서 본 거 말고는 별로
모델 출력: -1 / 실제 라벨: -1

# 4
리뷰: 이팝나무도서관 예약하고 갔음.
추운데 관계자 없어서 밖에서 떨면서 어찌저찌 연락되어 문 열어주면서 미안하단 말도 없었음
5분쯤 관람했을까? 홍보동영상 찍는다고 찍혀도 괜찮냐 물어서 싫다니 뒷쪽 방 열어줌
그곳에서 50분쯤 책 읽는데 안끝나겠다 싶어서 나옴
전주에 볼거 먹을거 많은데 일부러 시간내서 간게 너무 화가났음
비추
모델 출력: -1 / 실제 라벨: -1

# 5
리뷰: 왜 만든건지
모델 출력: 부정적인 댓글: -1 / 실제 라벨: -1

# 6
리뷰: 예술적 감각이 없는건지 모르겠다만 정말 이해도 전혀 안가고 그냥 폐공장일 뿐임
모델 출력: -1 / 실제 라벨: -1

# 7
리뷰: 사진찍기 안좋고 전시물도 없음
모델 출력: -1 / 실제 라벨: -1

# 8
리뷰: 볼게 하나도 없음
모델 출력: -1 / 실제 라벨: -1

# 9
리뷰: 볼게 하나도 없어서 넘 실망스러움
모델 출력: -1 / 실제 라벨: -1

# 10
리뷰: 별다른 경험이 없슴 기대 이하였네요
모델 출력: -1 / 실제 라벨: -1

# 11
리뷰: 너무불친절하세요
모델 출력: -1 / 실제 라벨: -1

# 12
리뷰: 휑한 팔복예술공장
모델 출력: -1 / 실제 라벨: -1

# 13
리뷰: 현대전시는 언젠나 재미없고 이해하기 어려움
모델 출력: -1 / 실제 라벨: -1

# 14
리뷰: 행사 등이 있을때 가는 곳이지 평일 주말에 관광 가는 곳은 아님
모델 출력: -1 / 실제 라벨: -1

# 15
리뷰: 포스터 전시가 그나마 나왔으나 전시 방법이 너무 아쉬움
모델 출력: -1 / 실제 

In [7]:
import pandas as pd
from langchain.chat_models import ChatOpenAI

# 데이터 불러오기
# data = pd.read_excel('/content/reviews.xlsx')
# print(data.head())

# 시스템 프롬프트는 동일하게 유지
system_prompt = '''
당신은 리뷰 분석 전문가입니다.
댓글을 읽고, 긍정적이면 1, 부정적이면 -1로 분류하세요.
반드시 정수로만 답하세요

---
'''

# 정확도 계산용 변수 초기화
correct_count = 0
total_count = 0

# [포인트] gpt-4 모델로 실행해 정확도 비교 가능
llm = ChatOpenAI(model="gpt-4", temperature=0.1)

# 리뷰 분석 및 결과 수집 루프
for i, row in data.iterrows():
    print(f'# {i+1}')
    print(f'리뷰: {row["Review"]}')

    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": row["Review"]}
    ]

    response = llm.invoke(prompt)
    result = response.content
    print(f'모델 출력: {result} / 실제 라벨: {row["Label"]}')

    if '-1' in result:
        prediction = -1
    else:
        prediction = 1

    if prediction == row["Label"]:
        correct_count += 1

    total_count += 1

    print()

# 정확도 출력
accuracy = correct_count / total_count
print(f'정확도: {accuracy:.2%}')

# 1
리뷰: 탄소와 예술의 결합 정말 별로
모델 출력: -1 / 실제 라벨: -1

# 2
리뷰: 크게 볼 것은 없습니다
모델 출력: -1 / 실제 라벨: -1

# 3
리뷰: 접근성 별로임. 차 있는 사람들에게 추천.
아이들과 같이 오는 공간.
이팝나무가 진 타이밍에 가서 그런지 볼게 없었음.
작품 전시회가 있어서 본 거 말고는 별로
모델 출력: -1 / 실제 라벨: -1

# 4
리뷰: 이팝나무도서관 예약하고 갔음.
추운데 관계자 없어서 밖에서 떨면서 어찌저찌 연락되어 문 열어주면서 미안하단 말도 없었음
5분쯤 관람했을까? 홍보동영상 찍는다고 찍혀도 괜찮냐 물어서 싫다니 뒷쪽 방 열어줌
그곳에서 50분쯤 책 읽는데 안끝나겠다 싶어서 나옴
전주에 볼거 먹을거 많은데 일부러 시간내서 간게 너무 화가났음
비추
모델 출력: -1 / 실제 라벨: -1

# 5
리뷰: 왜 만든건지
모델 출력: -1 / 실제 라벨: -1

# 6
리뷰: 예술적 감각이 없는건지 모르겠다만 정말 이해도 전혀 안가고 그냥 폐공장일 뿐임
모델 출력: -1 / 실제 라벨: -1

# 7
리뷰: 사진찍기 안좋고 전시물도 없음
모델 출력: -1 / 실제 라벨: -1

# 8
리뷰: 볼게 하나도 없음
모델 출력: -1 / 실제 라벨: -1

# 9
리뷰: 볼게 하나도 없어서 넘 실망스러움
모델 출력: -1 / 실제 라벨: -1

# 10
리뷰: 별다른 경험이 없슴 기대 이하였네요
모델 출력: -1 / 실제 라벨: -1

# 11
리뷰: 너무불친절하세요
모델 출력: -1 / 실제 라벨: -1

# 12
리뷰: 휑한 팔복예술공장
모델 출력: -1 / 실제 라벨: -1

# 13
리뷰: 현대전시는 언젠나 재미없고 이해하기 어려움
모델 출력: -1 / 실제 라벨: -1

# 14
리뷰: 행사 등이 있을때 가는 곳이지 평일 주말에 관광 가는 곳은 아님
모델 출력: -1 / 실제 라벨: -1

# 15
리뷰: 포스터 전시가 그나마 나왔으나 전시 방법이 너무 아쉬움
모델 출력: -1 / 실제 라벨: -1

#

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-wMdOK9OIhCKQQJqpQ18xuIQL on tokens per min (TPM): Limit 10000, Used 9950, Requested 82. Please try again in 192ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

## Langchain기반 댓글 감성분석 자동화(Langchain기반 모델과 템플릿 적용)

In [9]:
# ===================================== #
# 💡 감성 분석 자동화 시스템 (LangChain + GPT)
# ===================================== #

# 🧩 1. 기본 라이브러리 불러오기
import pandas as pd  # 엑셀 파일 데이터 읽기용
from langchain.prompts import PromptTemplate  # 템플릿 기반 프롬프트 구성 도구
from langchain.chat_models import ChatOpenAI  # OpenAI 기반 채팅 모델 연결
from langchain.schema import HumanMessage, SystemMessage  # 채팅 메시지 포맷 정의

# ============================== #
# [1단계] 리뷰 데이터 불러오기
# ============================== #

# 'reviews.xlsx' 파일에는 리뷰와 정답(긍정/부정 라벨)이 포함되어 있어야 합니다.


# ============================== #
# [2단계] 시스템 메시지 구성
# ============================== #

# [포인트] AI에게 역할과 규칙을 부여하는 부분입니다.
# 이 메시지는 AI가 '무엇을 해야 하는지'를 인식하게 만듭니다.
system_prompt = """
당신은 리뷰 분석 전문가입니다.
댓글을 읽고, 긍정적이면 1, 부정적이면 -1로 분류하세요.
반드시 정수로만 답하세요
"""

# ============================== #
# [3단계] 프롬프트 템플릿 정의
# ============================== #

# [포인트] PromptTemplate은 텍스트의 '형식'을 정의하는 도구입니다.
# 즉, 리뷰를 넣었을 때 어떤 구조로 AI에게 질문할지를 정해주는 역할을 합니다.
# 예: "당신은 리뷰 분석 전문가입니다. 리뷰: {review}"
prompt_template = PromptTemplate(
    input_variables=["review"],  # 템플릿에 넣을 변수: review (리뷰 텍스트)
    template="""
    {system_prompt}
    리뷰: {review}
    """
)

# ============================== #
# [4단계] 정확도 계산 변수 초기화
# ============================== #

correct_count = 0  # 맞은 예측 개수
total_count = 0    # 전체 리뷰 수

# ============================== #
# [5단계] LLM 모델 초기화 (GPT-4)
# ============================== #

# [포인트] GPT-4 모델을 사용하며, temperature는 0.1로 설정해 응답의 일관성을 높입니다.
# temperature는 AI의 창의성 정도를 조절하는 옵션입니다 (낮을수록 더 예측 가능하고 안정적).
llm = ChatOpenAI(model="gpt-4", temperature=0.1)

# ============================== #
# [6단계] 각 리뷰에 대해 분류 실행
# ============================== #

# [포인트] 리뷰 하나하나를 GPT에게 전달하고, 예측 결과를 비교합니다.
for i, row in data.iterrows():  # 한 줄씩 반복 처리 (i: 번호, row: 데이터 행)

    print(f'# {i+1}')  # 현재 리뷰 번호 출력
    print(f'리뷰: {row["Review"]}')  # 실제 리뷰 내용 출력

    # ✏️ 템플릿을 활용해 프롬프트 구성
    prompt = prompt_template.format(system_prompt=system_prompt, review=row["Review"])

    # [포인트] LangChain 형식에 맞춘 메시지 리스트 구성
    # SystemMessage → 시스템 역할 (분석가)
    # HumanMessage → 실제 유저 입력 (리뷰)
    messages = [
        SystemMessage(content=system_prompt),       # 시스템 역할 메시지
        HumanMessage(content=row["Review"])         # 유저 리뷰 입력
    ]

    # 💬 AI 모델에게 메시지를 보내고 응답 받기
    response = llm(messages)

    # 응답 결과(문자열) 저장
    result = response.content

    # 예측 결과와 실제 정답을 비교하여 출력
    print(f'모델 출력: {result} / 실제 라벨: {row["Label"]}')

    # ============================== #
    # [7단계] 예측값 처리 및 정답 비교
    # ============================== #

    # [포인트] '-1'이라는 문자열이 포함돼 있으면 부정으로 판단
    # 그 외의 경우는 모두 긍정으로 간주합니다
    if '-1' in result:
        prediction = -1
    else:
        prediction = 1

    # 예측값과 실제 라벨이 같다면 correct_count 증가
    if prediction == row["Label"]:
        correct_count += 1

    # 전체 처리 수 증가
    total_count += 1

    print()  # 구분선 출력용

# ============================== #
# [8단계] 전체 정확도 계산 및 출력
# ============================== #

# 정확도 = 맞은 수 / 전체 수
accuracy = correct_count / total_count

# 소수점 2자리까지 정확도 출력 (예: 92.31%)
print(f'정확도: {accuracy:.2%}')


# 1
리뷰: 탄소와 예술의 결합 정말 별로


<ipython-input-9-1011a1534428>:82: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


모델 출력: -1 / 실제 라벨: -1

# 2
리뷰: 크게 볼 것은 없습니다
모델 출력: -1 / 실제 라벨: -1

# 3
리뷰: 접근성 별로임. 차 있는 사람들에게 추천.
아이들과 같이 오는 공간.
이팝나무가 진 타이밍에 가서 그런지 볼게 없었음.
작품 전시회가 있어서 본 거 말고는 별로
모델 출력: -1 / 실제 라벨: -1

# 4
리뷰: 이팝나무도서관 예약하고 갔음.
추운데 관계자 없어서 밖에서 떨면서 어찌저찌 연락되어 문 열어주면서 미안하단 말도 없었음
5분쯤 관람했을까? 홍보동영상 찍는다고 찍혀도 괜찮냐 물어서 싫다니 뒷쪽 방 열어줌
그곳에서 50분쯤 책 읽는데 안끝나겠다 싶어서 나옴
전주에 볼거 먹을거 많은데 일부러 시간내서 간게 너무 화가났음
비추
모델 출력: -1 / 실제 라벨: -1

# 5
리뷰: 왜 만든건지
모델 출력: -1 / 실제 라벨: -1

# 6
리뷰: 예술적 감각이 없는건지 모르겠다만 정말 이해도 전혀 안가고 그냥 폐공장일 뿐임
모델 출력: -1 / 실제 라벨: -1

# 7
리뷰: 사진찍기 안좋고 전시물도 없음
모델 출력: -1 / 실제 라벨: -1

# 8
리뷰: 볼게 하나도 없음
모델 출력: -1 / 실제 라벨: -1

# 9
리뷰: 볼게 하나도 없어서 넘 실망스러움
모델 출력: -1 / 실제 라벨: -1

# 10
리뷰: 별다른 경험이 없슴 기대 이하였네요
모델 출력: -1 / 실제 라벨: -1

# 11
리뷰: 너무불친절하세요
모델 출력: -1 / 실제 라벨: -1

# 12
리뷰: 휑한 팔복예술공장
모델 출력: -1 / 실제 라벨: -1

# 13
리뷰: 현대전시는 언젠나 재미없고 이해하기 어려움
모델 출력: -1 / 실제 라벨: -1

# 14
리뷰: 행사 등이 있을때 가는 곳이지 평일 주말에 관광 가는 곳은 아님
모델 출력: -1 / 실제 라벨: -1

# 15
리뷰: 포스터 전시가 그나마 나왔으나 전시 방법이 너무 아쉬움
모델 출력: -1 / 실제 라벨: -1

# 16
리뷰: 주방아이템도 마켓형식으로 파는데